In [ ]:
from __future__ import print_function

import numpy as np
import SimpleITK as sitk
import six
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
from radiomics import firstorder, getTestCase, glcm, glrlm, glszm, imageoperations, shape

os.environ['CUDA_VISIABLEDIVCE']='4'

In [ ]:
# testBinWidth = 25 this is the default bin size
# testResampledPixelSpacing = [3,3,3] no resampling for now.

# Get some test data

# Download the test case to temporary files and return it's location. If already downloaded, it is not downloaded again,
# but it's location is still returned.
# imageName, maskName = getTestCase('brain1')    ### load nrrd image&label

# if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
#   print('Error getting testcase!')
#   exit()
def getFeatureFromImageRegion(imageName):
    image = sitk.ReadImage(imageName)
    img_array= sitk.GetArrayFromImage(image)
#     print(img_array.shape)
    #     mask = sitk.ReadIma
    # image = sitk.ReadImage('../../data/RawData/abnormal_png/abnormal_000.png')
    #     mask = sitk.ReadImage(maskName)
    mask = np.ones((img_array.shape[0],img_array.shape[1]), dtype=np.bool)
#     print(mask)
    mask = np.uint8(mask) ####cropTumor is multiplying image to mask
#     print(mask)
    mask_roi = sitk.GetImageFromArray(mask, isVector=False)
#     image = sitk.GetImageFromArray(image_img, isVector=False)
    
    applyLog = False
    applyWavelet = False

    # Setting for the feature calculation.
    # Currently, resampling is disabled.
    # Can be enabled by setting 'resampledPixelSpacing' to a list of 3 floats (new voxel size in mm for x, y and z)
    settings = {'binWidth': 25,
                'interpolator': sitk.sitkBSpline,
                'resampledPixelSpacing': None}

    #
    # If enabled, resample image (resampled image is automatically cropped.
    #
    
    interpolator = settings.get('interpolator')
    resampledPixelSpacing = settings.get('resampledPixelSpacing')
    if interpolator is not None and resampledPixelSpacing is not None:
        image, mask = imageoperations.resampleImage(image, mask, **settings)

    bb, correctedMask = imageoperations.checkMask(image, mask_roi)
    if correctedMask is not None:
        mask = correctedMask
    image, mask = imageoperations.cropToTumorMask(image, mask_roi, bb)
    img_check=sitk.GetArrayFromImage(image)
#     plt.imshow(img_check)
    #
    # Show the first order feature calculations
    #
    firstOrderFeatures = firstorder.RadiomicsFirstOrder(image, mask, **settings)

    firstOrderFeatures.enableFeatureByName('Mean', True)
    # firstOrderFeatures.enableAllFeatures()

    print('Calculating first order features...')
    results = firstOrderFeatures.execute()
    print('done')

    print('Calculated first order features: ')
    for (key, val) in six.iteritems(results):
        print('  ', key, ':', val)

    #
    # Show GLCM features
    #
    glcmFeatures = glcm.RadiomicsGLCM(image, mask, **settings)
    glcmFeatures.enableAllFeatures()

    print('Calculating GLCM features...')
    results = glcmFeatures.execute()
    print('done')

    print('Calculated GLCM features: ')
    for (key, val) in six.iteritems(results):
        print('  ', key, ':', val)

    #
    # Show GLRLM features
    #
    glrlmFeatures = glrlm.RadiomicsGLRLM(image, mask, **settings)
    glrlmFeatures.enableAllFeatures()

    print('Calculating GLRLM features...')
    results = glrlmFeatures.execute()
    print('done')

    print('Calculated GLRLM features: ')
    for (key, val) in six.iteritems(results):
        print('  ', key, ':', val)

    #
    # Show GLSZM features
    #
    glszmFeatures = glszm.RadiomicsGLSZM(image, mask, **settings)
    glszmFeatures.enableAllFeatures()

    print('Calculating GLSZM features...')
    results = glszmFeatures.execute()
#     print('done')

    print('Calculated GLSZM features: ')
    for (key, val) in six.iteritems(results):
        print('  ', key, ':', val)

In [ ]:
all_results=[]
one_results=[]
def getFeatureFromImageRegion(imageName):

    image = sitk.ReadImage(imageName)
    img_array= sitk.GetArrayFromImage(image)
#     print(img_array.shape)
    #     mask = sitk.ReadIma
    # image = sitk.ReadImage('../../data/RawData/abnormal_png/abnormal_000.png')
    #     mask = sitk.ReadImage(maskName)
    mask = np.ones((img_array.shape[0],img_array.shape[1]), dtype=np.bool)
#     print(mask)
    mask = np.uint8(mask) ####cropTumor is multiplying image to mask
#     print(mask)
    mask_roi = sitk.GetImageFromArray(mask, isVector=False)
#     image = sitk.GetImageFromArray(image_img, isVector=False)
    
    applyLog = False
    applyWavelet = False

    # Setting for the feature calculation.
    # Currently, resampling is disabled.
    # Can be enabled by setting 'resampledPixelSpacing' to a list of 3 floats (new voxel size in mm for x, y and z)
    settings = {'binWidth': 25,
                'interpolator': sitk.sitkBSpline,
                'resampledPixelSpacing': None}

    #
    # If enabled, resample image (resampled image is automatically cropped.
    #
    
    interpolator = settings.get('interpolator')
    resampledPixelSpacing = settings.get('resampledPixelSpacing')
    if interpolator is not None and resampledPixelSpacing is not None:
        image, mask = imageoperations.resampleImage(image, mask, **settings)

    bb, correctedMask = imageoperations.checkMask(image, mask_roi)
    if correctedMask is not None:
        mask = correctedMask
    image, mask = imageoperations.cropToTumorMask(image, mask_roi, bb)
    firstOrderFeatures = firstorder.RadiomicsFirstOrder(image, mask, **settings)

    #firstOrderFeatures.enableFeatureByName('Mean', True)
    firstOrderFeatures.enableAllFeatures()                     #############firstorder전부

    #print('Will calculate the following first order features: ')
    '''for f in firstOrderFeatures.enabledFeatures.keys():
        #print('  ', f)
        #print(getattr(firstOrderFeatures, 'get%sFeatureValue' % f).__doc__)'''

    #print('Calculating first order features...')
    results = firstOrderFeatures.execute()
    #print('done')
    
    results_first_order = {}
    for (key, val) in six.iteritems(results):
        results_first_order.update({key : val})

    csv_first_order = pd.DataFrame.from_dict(results_first_order, orient='index')
    csv_first_order.columns = ['Value']
    csv_first_order.reset_index(level=0, inplace=True)
    csv_first_order['Feature_ID'] = 'First_Order'
    cols = csv_first_order.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    csv_first_order = csv_first_order[cols]
    FOlist= csv_first_order['Feature_ID']+'_'+csv_first_order['index']
    FOvalue= csv_first_order['Value']
    
    #
    # Show GLCM features   24
    #
    glcmFeatures = glcm.RadiomicsGLCM(image, mask, **settings)                            ##############GLCM
    glcmFeatures.enableAllFeatures()

    #print('Will calculate the following GLCM features: ')
    #for f in glcmFeatures.enabledFeatures.keys():
        #print('  ', f)
        #print(getattr(glcmFeatures, 'get%sFeatureValue' % f).__doc__)

    #print('Calculating GLCM features...')
    results = glcmFeatures.execute()
    results_GLCM = {}
    for (key, val) in six.iteritems(results):
        results_GLCM.update({key : val})
    #print('done')

    csv_GLCM = pd.DataFrame.from_dict(results_GLCM, orient='index')
    csv_GLCM.columns = ['Value']
    csv_GLCM.reset_index(level=0, inplace=True)
    csv_GLCM['Feature_ID'] = 'GLCM'
    cols = csv_GLCM.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    csv_GLCM = csv_GLCM[cols]
#     print(csv_GLCM)
    GLCMlist= csv_GLCM['Feature_ID']+'_'+csv_GLCM['index']
    GLCMvalue= csv_GLCM['Value']
    #
    # Show GLRLM features     16
    #
    glrlmFeatures = glrlm.RadiomicsGLRLM(image, mask, **settings)
    glrlmFeatures.enableAllFeatures()

    #print('Will calculate the following GLRLM features: ')
    #for f in glrlmFeatures.enabledFeatures.keys():
        #print('  ', f)
        #print(getattr(glrlmFeatures, 'get%sFeatureValue' % f).__doc__)

    #print('Calculating GLRLM features...')
    results = glrlmFeatures.execute()
    #print('done')
    results_GLRLM = {}
    for (key, val) in six.iteritems(results):
        results_GLRLM.update({key : val})
        
    csv_GLRLM = pd.DataFrame.from_dict(results_GLRLM, orient='index')
    csv_GLRLM.columns = ['Value']
    csv_GLRLM.reset_index(level=0, inplace=True)
    csv_GLRLM['Feature_ID'] = 'GLRLM'
    cols = csv_GLRLM.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    
    csv_GLRLM = csv_GLRLM[cols]
    GLRLMlist= csv_GLRLM['Feature_ID']+'_'+csv_GLRLM['index']
    GLRLMvalue= csv_GLRLM['Value']
    #
    # Show GLSZM features   16
    #L
    glszmFeatures = glszm.RadiomicsGLSZM(image, mask, **settings)
    glszmFeatures.enableAllFeatures()

    #print('Will calculate the following GLSZM features: ')
    #for f in glszmFeatures.enabledFeatures.keys():
        #print('  ', f)
        #print(getattr(glszmFeatures, 'get%sFeatureValue' % f).__doc__)

    #print('Calculating GLSZM features...')
    results = glszmFeatures.execute()
    results_GLSZM = {}
    for (key, val) in six.iteritems(results):
        results_GLSZM.update({key : val})
    #print('done')

    csv_GLSZM = pd.DataFrame.from_dict(results_GLSZM, orient='index')
    csv_GLSZM.columns = ['Value']
    csv_GLSZM.reset_index(level=0, inplace=True)
    csv_GLSZM['Feature_ID'] = 'GLSZM'
    cols = csv_GLSZM.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    csv_GLSZM = csv_GLSZM[cols]
    GLSZMlist= csv_GLSZM['Feature_ID']+'_'+csv_GLSZM['index']
    GLSZMvalue= csv_GLSZM['Value']
         
    frames = [csv_first_order, csv_GLCM, csv_GLRLM, csv_GLSZM]
    frames_list = [FOlist, GLCMlist, GLRLMlist, GLSZMlist]
    frames_value= [FOvalue, GLCMvalue, GLRLMvalue, GLSZMvalue]
    
    All_results = pd.concat(frames)
    All_results = All_results.T
    res_csv = pd.DataFrame.from_dict(All_results)
#     res_dir = ('../featureData/20201227/train_normal/' + name.split("/")[-1][:-4] + '.csv')
#     res_dir = ('../featureData/nonfilter/origin_size/each/ab_' + str(name)+ '.csv')
    
    All_list=pd.concat(frames_list)
    #     All_list = All_list.T
    All_list = pd.DataFrame.from_dict(All_list)
    All_list = All_list.transpose()
    
    All_value=pd.concat(frames_value)
    All_value = All_value.transpose()
    All_value = All_value.reset_index(drop=True)
    all_results.append(All_value)
#     one_results.append(All_value)
#     print(len(all_results))
    
#     All_value = pd.DataFrame.from_dict(All_value)
#     All_value = All_value.transpose()
    
    
    
#     res_csv.to_csv(res_dir, index=False, na_rep='NaN')
 ############list   
    All_list.to_csv('../featureData/20201227/All_list.csv', index=False, na_rep='NaN')
    return All_list, all_results    ###############result concatnate
#     print(name.split("/")[-1][:-4] + ' : Done')

    

In [ ]:
image = sitk.ReadImage('../../data/RawData/abnormal_png/abnormal_000.png')
# print(sitk.Image.GetSize)
img_array= sitk.GetArrayFromImage(image)
print(img_array.shape)

In [ ]:
getFeatureFromImageRegion('../featureData/sameRegion/32/abnormal_all/abnormal_000.png')

In [ ]:
print(len(ab_path), ab_path[0])
print(len(nor_path), nor_path[0])

In [ ]:
train_ab_path= glob.glob('../featureData/20201227/data/train/abnormal/*.png')
train_nor_path= glob.glob('../featureData/20201227/data/train/normal/*.png')
test_ab_path= glob.glob('../featureData/20201227/data/test/abnormal/*.png')
test_nor_path=glob.glob('../featureData/20201227/data/test/normal/*.png')

print(len(train_ab_path),len(train_nor_path),len(test_ab_path),len(test_nor_path))

In [ ]:
# for name in train_nor_path:

#     getFeatureFromImageRegion(name)
    
# results= pd.DataFrame(all_results)
# results.to_csv('../featureData/20201227/data/train_normal_all.csv', index=False, na_rep='NaN')
# one_results=[]

# for name in train_ab_path:

#     getFeatureFromImageRegion(name)
    
# results= pd.DataFrame(all_results)
# results.to_csv('../featureData/20201227/data/train_abnormal_all.csv', index=False, na_rep='NaN')


# for name in test_nor_path:

#     getFeatureFromImageRegion(name)
    
# results= pd.DataFrame(all_results)
# results.to_csv('../featureData/20201227/data/test_normal_all.csv', index=False, na_rep='NaN')
all_results=[]

for name in test_ab_path:

    getFeatureFromImageRegion(name)
    
results= pd.DataFrame(all_results)
results.to_csv('../featureData/20201227/data/test_abnormal_all.csv', index=False, na_rep='NaN')

# results= pd.DataFrame(all_results)
# results.to_csv('../featureData/20201227/data/results_all_sort.csv', index=False, na_rep='NaN')

In [ ]:
# print(all_results.type)
results= pd.DataFrame(all_results)
results.to_csv('../featureData/32_feature/All/abnormal_results.csv', index=False, na_rep='NaN')